In [ ]:
#@title Install to Google Drive (for Resuming Training & Automatic Saving)
#@markdown <small> This Notebook is based on another found in: https://github.com/ardha27/AI-Song-Cover-RVC < Visit this repo to read more and support.
%cd /content
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import clear_output
from ipywidgets import Button
import os
if not os.path.exists('/content/drive'):
    print("Your drive is not mounted. Creating Fake Drive.")
    os.makedirs('/content/drive/MyDrive')
source = "Rejekts"
!wget https://huggingface.co/{source}/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip -n 'project-main.zip' -d /content/drive/MyDrive
!cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements-safe.txt'
!pip install pyngrok
!rm /content/project-main.zip
!rm -r /content/sample_data
!mkdir -p /content/dataset
clear_output()
Button(description="\u2714 Success", button_style="success")

In [ ]:
#@title 1.Preprocess Data
%cd /content/drive/MyDrive/project-main
model_name = 'elon' #{type:"string"}
# <small> Enter the path to your dataset folder (a folder with audios of the vocals you will train on), or if you want just upload the audios using the File Manager into the 'dataset' folder.
dataset_folder = '/content/drive/MyDrive/dataset'
while len(os.listdir(dataset_folder)) < 1:
    input("Your dataset folder is empty.")
!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Starting...")
!python infer/modules/train/preprocess.py {dataset_folder} 40000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your dataset folder is correct.")

In [ ]:
#@title 2.Extract Features
f0method = "rmvpe_gpu" #  ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
%cd /content/drive/MyDrive/project-main
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting...")
if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 ./logs/{model_name} v2
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your data was preprocessed.")

In [ ]:
#@title 3.Train Index
import numpy as np
import faiss
%cd /content/drive/MyDrive/project-main
def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))

In [ ]:
#@title 4.Train Model
#@markdown <small> Enter your ngrok authtoken to open tensorboard. Get one here: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_authtoken = "2OkBvQxPQw7QeiCZMUlszGMX4Jq_2LLKJDWYr1WG7nGRLD85R"#@param {type:"string"}
!ngrok config add-authtoken {ngrok_authtoken}
#%cd /content/drive/MyDrive/project-main
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
from pyngrok import ngrok
now_dir=os.getcwd()
#@markdown <small> Enter the name of your model again. It must be the same you chose before.
model_name = 'elon'#@param {type:"string"}
#@markdown <small> Choose how often to save the model and how much training you want it to have.
save_frequency = 20 # @param {type:"slider", min:5, max:50, step:5}
epochs = 300 # @param {type:"slider", min:10, max:1000, step:10}
#@markdown <small> ONLY cache datasets under 10 minutes long. Otherwise leave this unchecked.
cache = True #@param {type:"boolean"}
# Remove the logging setup

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
print("Tensorboard NGROK URL:",end="")
try:
    training_log = click_train(
        model_name,
        '40k',
        True,
        0,
        save_frequency,
        epochs,
        12,
        True,
        'assets/pretrained_v2/f0G40k.pth',
        'assets/pretrained_v2/f0D40k.pth',
        0,
        cache,
        True,
        'v2',
    )
    print(training_log)
except:
    ngrok.kill()

In [ ]:
#@title Inference
%cd /content/drive/MyDrive/project-main
transpose = 0#@param {type:"integer"}
input_path = "/content/drive/MyDrive/harvard.wav"#@param {type:"string"}
index_path = "/logs/elon/added_IVF749_Flat_nprobe_1_elon_v2.index"#@param {type:"string"}
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"]
opt_path = "/audios/cli_output.wav"#@param {type:"string"}
model_name = "elon.pth"#@param {type:"string"}
index_rate = 0.63 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0.13 # @param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.33 # @param {type:"slider", min:0, max:1, step:0.01}

#!rm $opt_path
!python tools/infer_cli.py --f0up_key $transpose \
--input_path $input_path \
--index_path $index_path \
--f0method $f0_method \
--opt_path $opt_path \
--model_name $model_name \
--index_rate $index_rate \
--device cuda:0 \
--is_half True \
--filter_radius 3 \
--resample_sr 0 \
--rms_mix_rate $volume_normalization \
--protect $consonant_protection

import IPython.display as ipd

#ipd.clear_output()
sample_rate = 40000

ipd.Audio(opt_path, rate=sample_rate)


In [ ]:
import json
import sys
import wave
from pathlib import Path

BASE_DIR = Path('/content')
sys.path.append(str(BASE_DIR))

import torch
from multiprocessing import cpu_count
from flask_ngrok3 import run_with_ngrok
from flask import Flask, request, send_file

from vc_infer_pipeline import VC
from infer_pack.models import (
    SynthesizerTrnMs256NSFsid,
    SynthesizerTrnMs256NSFsid_nono,
    SynthesizerTrnMs768NSFsid,
    SynthesizerTrnMs768NSFsid_nono,
)
from my_utils import load_audio
from fairseq import checkpoint_utils
from scipy.io import wavfile


INPUT_VOICE_PATH = 'input.mp3'
OUTPUT_VOICE_PATH = 'output.wav'
MODEL_NAME = 'elon'
DEVICE = 'cuda:0'
cpt = None


class Config:
  def __init__(self, device, is_half):
    self.device = device
    self.is_half = is_half
    self.n_cpu = 0
    self.gpu_name = None
    self.gpu_mem = None
    self.x_pad, self.x_query, self.x_center, self.x_max = self.device_config()

  def device_config(self) -> tuple:
    if torch.cuda.is_available():
      i_device = int(self.device.split(":")[-1])
      self.gpu_name = torch.cuda.get_device_name(i_device)

      if (
        ("16" in self.gpu_name and "V100" not in self.gpu_name.upper())
        or "P40" in self.gpu_name.upper()
        or "1060" in self.gpu_name
        or "1070" in self.gpu_name
        or "1080" in self.gpu_name
      ):
        print("16 series/10 series P40 forced single precision")
        self.is_half = False
        for config_file in ["32k.json", "40k.json", "48k.json"]:
          with open(f"configs/{config_file}", "r") as f:
            strr = f.read().replace("true", "false")
          with open(f"configs/{config_file}", "w") as f:
            f.write(strr)
        with open("trainset_preprocess_pipeline_print.py", "r") as f:
          strr = f.read().replace("3.7", "3.0")
        with open("trainset_preprocess_pipeline_print.py", "w") as f:
          f.write(strr)
      else:
        self.gpu_name = None

      self.gpu_mem = int(
        torch.cuda.get_device_properties(i_device).total_memory
        / 1024
        / 1024
        / 1024
        + 0.4
      )
      if self.gpu_mem <= 4:
        with open("trainset_preprocess_pipeline_print.py", "r") as f:
          strr = f.read().replace("3.7", "3.0")
        with open("trainset_preprocess_pipeline_print.py", "w") as f:
          f.write(strr)

    elif torch.backends.mps.is_available():
      print("No supported N-card found, use MPS for inference")
      self.device = "mps"
    else:
      print("No supported N-card found, use CPU for inference")
      self.device = "cpu"
      self.is_half = True

    if self.n_cpu == 0:
      self.n_cpu = cpu_count()

    if self.is_half:
      # 6G memory config
      x_pad = 3
      x_query = 10
      x_center = 60
      x_max = 65
    else:
      # 5G memory config
      x_pad = 1
      x_query = 6
      x_center = 38
      x_max = 41

    if self.gpu_mem != None and self.gpu_mem <= 4:
      x_pad = 1
      x_query = 5
      x_center = 30
      x_max = 32

    return x_pad, x_query, x_center, x_max


CONFIG = Config(DEVICE, True)


def load_hubert():
  models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task(['hubert_base.pt'], suffix='', )
  hubert = models[0]
  hubert = hubert.to(DEVICE)

  if True:
    hubert = hubert.half()
  else:
    hubert = hubert.float()

  hubert.eval()
  return hubert

HUBERT_MODEL = load_hubert()


def get_vc(device, is_half, config):
  global cpt, version, net_g, tgt_sr, vc
  model_path = BASE_DIR / 'weights' / f'{MODEL_NAME}.pth'
  if not model_path.exists():
    print(f'The model {model_path} does not exist. Please ensure that you have filled in the proper MODEL_NAME in your .env file.')
    return None

  model_path = str(model_path)
  print(f'loading pth {model_path}')
  cpt = torch.load(model_path, map_location='cpu')
  tgt_sr = cpt["config"][-1]
  cpt["config"][-3] = cpt["weight"]["emb_g.weight"].shape[0]
  if_f0 = cpt.get("f0", 1)
  version = cpt.get("version", "v1")

  if version == "v1":
    if if_f0 == 1:
      net_g = SynthesizerTrnMs256NSFsid(*cpt["config"], is_half=is_half)
    else:
      net_g = SynthesizerTrnMs256NSFsid_nono(*cpt["config"])
  elif version == "v2":
    if if_f0 == 1:
      net_g = SynthesizerTrnMs768NSFsid(*cpt["config"], is_half=is_half)
    else:
      net_g = SynthesizerTrnMs768NSFsid_nono(*cpt["config"])

  del net_g.enc_q
  print(net_g.load_state_dict(cpt["weight"], strict=False))
  net_g.eval().to(device)

  if is_half:
    net_g = net_g.half()
  else:
    net_g = net_g.float()

  vc = VC(tgt_sr, config)


def rvc_infer(pitch_change, pitch_extraction_algo, volume_envelope, index_rate):
  logs_dir = BASE_DIR / 'logs' / MODEL_NAME
  index_path = ''
  for file in logs_dir.iterdir():
    if file.suffix == '.index':
      index_path = str(logs_dir / file.name)
      break

  # vc single
  audio = load_audio(INPUT_VOICE_PATH, 16000)
  times = [0, 0, 0]
  if_f0 = cpt.get('f0', 1)
  audio_opt = vc.pipeline(HUBERT_MODEL, net_g, 0, audio, INPUT_VOICE_PATH, times, pitch_change, pitch_extraction_algo, index_path, index_rate, if_f0, 3, tgt_sr, 0, volume_envelope, version, 0.33, f0_file=None)
  wavfile.write(OUTPUT_VOICE_PATH, tgt_sr, audio_opt)


app = Flask(__name__)
run_with_ngrok(app, auth_token=NGROK_AUTH_TOKEN)

@app.route('/', methods=['GET'])
def test():
  response = {'status':'OK','message':'Test'}
  return json.dumps(response)


@app.route('/infer', methods=['POST'])
def infer():
  global MODEL_NAME, cpt
  model_name = request.args.get('model')
  if MODEL_NAME != model_name:
    MODEL_NAME = model_name
    if cpt:
      del cpt
    get_vc(DEVICE, True, CONFIG)

  pitch_change = int(request.args.get('pitch'))
  pitch_extraction_algo = request.args.get('algo')
  volume_envelope = float(request.args.get('volume'))
  index_rate = float(request.args.get('index_rate'))
  audio_data = request.files['audio_file']
  audio_data.save(INPUT_VOICE_PATH)
  rvc_infer(pitch_change, pitch_extraction_algo, volume_envelope, index_rate)
  return send_file(OUTPUT_VOICE_PATH, mimetype="audio/wav")

app.run()

In [ ]:
#@title OR Open the GUI (Banned for free Colab Notebooks)
if not 'installed' in locals():
    %cd /content
    from google.colab import drive
    drive.mount('/content/drive')
    from IPython.display import clear_output
    from ipywidgets import Button
    import os
    if not os.path.exists('/content/drive'):
        print("Your drive is not mounted. Creating Fake Drive.")
        os.makedirs('/content/drive/MyDrive')
    if not os.path.exists('/content/drive/MyDrive/project-main'):
        !wget https://huggingface.co/Rejekts/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip 'project-main.zip' -d /content/drive/MyDrive
    !cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements.txt'
    !rm /content/project-main.zip
    !rm -r /content/sample_data
    !mkdir -p /content/dataset
    clear_output()
    Button(description="\u2714 Success", button_style="success")
tensorboard = True #@param {type:"boolean"}
if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs
%cd /content/drive/MyDrive/project-main
!python app.py --colab